In [1]:
from google.colab import files
uploaded = files.upload()

Saving out.csv to out.csv


In [4]:
from google.colab import files
uploaded = files.upload()

Saving final_data.csv to final_data.csv


In [5]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
import re

In [6]:
from sklearn import datasets
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV, cross_val_score
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [7]:
import pandas as pd
data = pd.read_csv("out.csv")

In [8]:
data

,Unnamed: 0,IsToxic,IsAbusive,IsThreat,IsProvocative,IsObscene,IsHatespeech,IsRacist,IsNationalist,IsSexist,IsReligiousHate,LEMM
0,0,False,False,False,False,False,False,False,False,False,False,"If people would take step back make case , ' a..."
1,1,True,True,False,False,False,False,False,False,False,False,Law enforcement trained shoot apprehend . They...
2,2,True,True,False,False,True,False,False,False,False,False,Dont reckon 'black life matter ' banner held w...
3,3,False,False,False,False,False,False,False,False,False,False,There large number people like police officer ...
4,4,False,False,False,False,False,False,False,False,False,False,"The Arab dude absolutely right , shot 6 extra ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,False,False,False,False,False,False,False,False,False,False,I remember sent national defence
996,996,True,False,False,False,False,True,True,False,False,False,Stats ` represent problem . Race baiting attit...
997,997,False,False,False,False,False,False,False,False,False,False,The quote mother ... Wow hit hard . Very accur...
998,998,False,False,False,False,False,False,False,False,False,False,video racist


In [9]:
data.drop(columns = ["Unnamed: 0"] , inplace = True)

In [10]:
Y = data['IsToxic']
X = data['LEMM']
X_train_bow, X_test_bow, Y_train , Y_test = train_test_split(X,Y)

In [11]:
data = pd.read_csv("final_data.csv")

In [12]:
data.drop(columns = ["Unnamed: 0", "Unnamed: 0.1", "Unnamed: 0.1.1"], inplace = True)

In [13]:
cols = data.columns
cols

Index(['IsToxic', 'IsAbusive', 'IsThreat', 'IsProvocative', 'IsObscene',
       'IsHatespeech', 'IsRacist', 'IsNationalist', 'IsSexist',
       'IsReligiousHate', 'LEMM'],
      dtype='object')

In [14]:
data_raw = pd.read_csv("out.csv")

In [15]:
data_raw = data_raw.dropna()
data = data.dropna()

In [31]:
def check_LR(data , i):
    raw = data.iloc[0:1000,:]
    aug = data.iloc[1000:,:]

    Y_raw = raw[cols[i]]
    X_raw = raw['LEMM']
    Y_aug = aug[cols[i]]
    X_aug = aug['LEMM']

    X_train_temp, X_test_temp, Y_train_temp , Y_test_temp = train_test_split(X_raw,Y_raw)

    X_test = X_test_temp
    Y_test = Y_test_temp

    X_aug = np.concatenate((X_aug , X_train_temp))
    Y_aug = np.concatenate((Y_aug , Y_train_temp))

    Y_train = Y_aug
    X_train_bow = X_aug
    X_test_bow = X_test
    Y_test = Y_test

# {'kernel': 'rbf', 'gamma': 'scale', 'C': 10.0}

    clf = LogisticRegression(max_iter = 100)
    clf2 = LogisticRegression(max_iter = 100)

    vectorizer = CountVectorizer()
    vectorizer.fit(X_train_bow)
    X_train_bow = vectorizer.transform(X_train_bow)
    X_test_bow = vectorizer.transform(X_test_bow)

    X_train_bow = X_train_bow.toarray()
    # Y_train = Y_train.to_numpy()
    X_test_bow = X_test_bow.toarray()
    Y_test = Y_test.to_numpy() 

    X_train_tf = X_aug
    # Y_train = Y_train
    X_test_tf = X_test
    # Y_test = Y_test

    # Tf-IDF generator
    vectorizer_tf = TfidfVectorizer(lowercase=True,stop_words='english')
    vectorizer_tf.fit(X_train_tf)
    X_train_tf = vectorizer.transform(X_train_tf)
    X_test_tf = vectorizer.transform(X_test_tf)

    X_temp_tf = X_train_tf.toarray()
    # Y_temp_tf = Y_train_tf.to_numpy()
    X_temp_test_tf = X_test_tf.toarray()
    # Y_temp_test_tf = Y_test_tf.to_numpy()

    # X_train -> X_train_tf
    # Y_train -> Y_train
    # X_test -> X_temp_test_tf
    # Y_test -> Y_test

    clf.fit(X_train_bow , Y_train)

    y_pred1 = clf.predict(X_test_bow)
    score1 = accuracy_score(y_pred1 , Y_test)

    clf2.fit(X_train_tf , Y_train)

    y_pred = clf2.predict(X_temp_test_tf)
    score = accuracy_score(y_pred , Y_test)

    return (score1,score)

In [32]:
Tfidf_aug = []
bow_aug = []
Tfidf = []
bow = []

for i in range(len(cols)-1):
    print(cols[i])
    s1 , s2 = check_LR(data , i)
    s3, s4 = check_LR(data_raw , i)
    bow_aug.append(s1)
    Tfidf_aug.append(s2)
    bow.append(s3)
    Tfidf.append(s4)

IsToxic
IsAbusive
IsThreat


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _che

IsProvocative
IsObscene
IsHatespeech
IsRacist
IsNationalist
IsSexist
IsReligiousHate


In [33]:
Tfidf_aug

[0.988, 1.0, 0.98, 0.992, 1.0, 0.996, 0.996, 0.996, 1.0, 1.0]

In [34]:
bow_aug

[0.988, 1.0, 0.98, 0.992, 1.0, 0.996, 0.996, 0.996, 1.0, 1.0]

In [35]:
Tfidf

[0.704, 0.74, 0.976, 0.88, 0.908, 0.864, 0.836, 0.996, 1.0, 0.976]

In [36]:
bow

[0.704, 0.74, 0.976, 0.88, 0.908, 0.864, 0.836, 0.996, 1.0, 0.976]

In [44]:
columns = [ "Label" , "Raw Data" , "Augmented Data"]
df_bow = []
df_tf = []

In [45]:
for i in range(10):
    ar1 = [cols[i],Tfidf[i],Tfidf_aug[i]]
    ar2 = [cols[i],bow[i],bow_aug[i]]
    df_tf.append(ar1)
    df_bow.append(ar2)

In [46]:
table_bow = pd.DataFrame(df_bow , columns = columns)
table_tf = pd.DataFrame(df_tf , columns = columns)

In [48]:
table_bow

,Label,Raw Data,Augmented Data
0,IsToxic,0.704,0.988
1,IsAbusive,0.740,1.000
2,IsThreat,0.976,0.980
3,IsProvocative,0.880,0.992
4,IsObscene,0.908,1.000
5,IsHatespeech,0.864,0.996
6,IsRacist,0.836,0.996
7,IsNationalist,0.996,0.996
8,IsSexist,1.000,1.000
9,IsReligiousHate,0.976,1.000


In [49]:
table_tf

,Label,Raw Data,Augmented Data
0,IsToxic,0.704,0.988
1,IsAbusive,0.740,1.000
2,IsThreat,0.976,0.980
3,IsProvocative,0.880,0.992
4,IsObscene,0.908,1.000
5,IsHatespeech,0.864,0.996
6,IsRacist,0.836,0.996
7,IsNationalist,0.996,0.996
8,IsSexist,1.000,1.000
9,IsReligiousHate,0.976,1.000
